In [1]:
import os
import argparse
import time
import json
import torch
import random
import torch.nn as nn
import numpy as np
from dataset_BERT import EurDataset, collate_data
from performance_BERT import performance
import logging
import sys 
sys.path.append("..") 
from models.BERT2FC import DeepSC_BERT2FC
from utils import SNR_to_noise, initNetParams, train_step_bart2fc, val_step_bart2fc, NoamOpt, EarlyStopping
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

parser = argparse.ArgumentParser()
parser.add_argument('--data-dir', default='../data/BERT/train_data.pkl', type=str)
parser.add_argument('--checkpoint-path', default='../checkpoints/BERT2FC/lr=1e-5', type=str)
parser.add_argument('--channel', default='TEST', type=str, help='Please choose AWGN, Rayleigh, and Rician')
parser.add_argument('--MAX-LENGTH', default=70, type=int)
parser.add_argument('--batch-size', default=32, type=int)
parser.add_argument('--epochs', default=400, type=int)
parser.add_argument('--resume', default=False, type=bool)
parser.add_argument('--Test_epochs', default=1, type=int)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

log_format = '%(asctime)s %(message)s'
logging.basicConfig(filename='myLog.log', 
                    filemode='w',
                    level=logging.INFO,
                    format=log_format,
                    datefmt='%m/%d %I:%M:%S %p')
logger = logging.getLogger()

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


def validate(epoch, args, net, noise_std):
    test_eur = EurDataset('test')
    test_iterator = DataLoader(test_eur, batch_size=args.batch_size, num_workers=0,
                               pin_memory=True, collate_fn=collate_data)
    pbar = tqdm(test_iterator)
    total = 0

    with torch.no_grad():
        for sents in pbar:
            sents = sents.to(device)
            loss = val_step_bart2fc(net, sents, sents, noise_std, pad_idx, criterion, args.channel)
            total += loss
            pbar.set_description('Epoch: {}; Type: Vaild; Loss: {:.5f}'.format(epoch, loss))
    
    early_stopping(total / len(test_iterator), net, logger)
    if early_stopping.early_stop:
        sys.exit("Early stopping")   

    return total / len(test_iterator)


def train(epoch, args, net, noise_std):
    train_eur = EurDataset('train')
    train_iterator = DataLoader(train_eur, batch_size=args.batch_size, num_workers=0, shuffle=True,
                                pin_memory=True, collate_fn=collate_data)
    pbar = tqdm(train_iterator)
    total = 0

    for sents in pbar:
        sents = sents.to(device)
        loss = train_step_bart2fc(net, sents, sents, noise_std, pad_idx, opt, criterion, args.channel)
        total += loss
        pbar.set_description('Epoch: {};  Type: Train; Loss: {:.5f}'.format(epoch, loss))
       
    return total / len(train_iterator)

if __name__ == '__main__':

    """ parameter setting"""
    setup_seed(42)
    args = parser.parse_args(args=[])
    early_stopping = EarlyStopping(args.checkpoint_path + '/best')
    loss_curve = SummaryWriter("../logs/BERT2FC/lr=1e-7", flush_secs=1)
    logger.info('The args: {}'.format(args))


    """ special token idx """
    vocab_size = 30522
    start_idx = 101
    pad_idx = 0
    end_idx = 102


    """ define model """
    deepsc_bert2fc = DeepSC_BERT2FC(vocab_size).to(device)


    """ load existed model"""
    if args.resume:
        model_paths = []
        for fn in os.listdir(args.checkpoint_path):
            if not fn.endswith('.pth'): continue
            idx = int(os.path.splitext(fn)[0].split('_')[-1])  # read the idx of image
            model_paths.append((os.path.join(args.checkpoint_path, fn), idx))

        model_paths.sort(key=lambda x: x[1])  # sort the image by the idx
        model_path, _ = model_paths[-1]
        print(model_path)
        checkpoint = torch.load(model_path, map_location='cpu')
        deepsc_bert2fc.load_state_dict(checkpoint,strict=False)
        print('model load!')
    else:
        print('no existed checkpoint')
        for p in deepsc_bert2fc.quantization.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p) 
        for p in deepsc_bert2fc.dequantization.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p) 
        for p in deepsc_bert2fc.dense.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p) 
    
    
    """ define optimizer and loss function """
    criterion = nn.CrossEntropyLoss(reduction='none')
    opt = torch.optim.Adam(deepsc_bert2fc.parameters(), lr=1e-6, betas=(0.9, 0.98), eps=1e-8, weight_decay=5e-4)
    # opt = NoamOpt(768, 1, 20000, optimizer)

    record_acc = 10
    for epoch in range(358 , 358 + args.epochs):

        start = time.time()
        n_std = np.random.uniform(SNR_to_noise(5), SNR_to_noise(10))

        tra_acc = train(epoch, args, deepsc_bert2fc, n_std)
        avg_acc = validate(epoch, args, deepsc_bert2fc, n_std)

        if record_acc >= avg_acc:
            record_acc = avg_acc
            if not os.path.exists(args.checkpoint_path):
                os.makedirs(args.checkpoint_path)
            with open(args.checkpoint_path + '/checkpoint_{}.pth'.format(str(epoch).zfill(2)), 'wb') as f:
                torch.save(deepsc_bert2fc.state_dict(), f)

        bleu1, bleu2, bleu3, bleu4 = performance(args, [0], deepsc_bert2fc, pad_idx, start_idx, end_idx)

        # record the results
        logger.info('Epoch: {}; Type: Train; Loss: {:.5f}'.format(epoch, tra_acc))  
        logger.info('Epoch: {}; Type: Vaild; Loss: {:.5f}'.format(epoch, avg_acc))
        logger.info('Epoch: {}; Type: Vaild; BLEU score: {:.5f} {:.5f} {:.5f} {:.5f}'.format(epoch, bleu1[0], bleu2[0], bleu3[0], bleu4[0]))

        loss_curve.add_scalar("Train loss", tra_acc, epoch)
        loss_curve.add_scalar("Vaild loss", avg_acc, epoch)
        loss_curve.add_scalar("BLEU score", bleu1[0], epoch)

    loss_curve.close()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


../checkpoints/BERT2FC/lr=1e-5\checkpoint_357.pth
model load!


Epoch: 358; Type: Vaild; Loss: 0.18393: 100%|██████████| 372/372 [00:14<00:00, 26.37it/s]


Validation loss decreased (inf --> 0.303180).  Saving model ...


Epoch: 359; Type: Vaild; Loss: 0.18310: 100%|██████████| 372/372 [00:13<00:00, 26.87it/s]


...

EarlyStopping counter: 9 out of 10 , Validation loss is 0.269592


Epoch: 491; Type: Vaild; Loss: 0.17758: 100%|██████████| 372/372 [00:12<00:00, 30.68it/s]

EarlyStopping counter: 10 out of 10 , Validation loss is 0.269612


SystemExit: Early stopping

c:\Users\stone4022\anaconda3\envs\pytorch38\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
